In [39]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
stop_words = set(stopwords.words('english')) 
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
import re , unicodedata , string

[nltk_data] Downloading package stopwords to /home/amanv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/amanv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/amanv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
df = pd.read_csv('./IndianFoodDatasetCSV.csv')
df = df.dropna()

#### Why do we need to remove punctuation
The punctuation removal process will help to treat each text equally. For example, the word data and data! are treated equally after the process of removal of punctuations.

In [41]:
# Remove Punctuation
def remove_punctuation(sample_str):
    """Remove punctuation from a sample string"""
    words = word_tokenize(sample_str)
    new_words = []
    for word in words:
      new_word = re.sub(r'[^\w\s]', '', word)
      if new_word != '':
        new_words.append(new_word)
    return ' '.join(new_words)

stopword_list = set(stopwords.words('english'))

def remove_stopwords(sample_str):
      """Remove stop words from a sample string"""
      words = word_tokenize(sample_str)
      new_words = []
      for word in words:
          if word.lower() not in stopword_list:
              new_words.append(word)
      return ' '.join(new_words)

df['TranslatedRecipeName'] = df['TranslatedRecipeName'].apply(remove_punctuation).apply(remove_stopwords)
df['TranslatedIngredients'] = df['TranslatedIngredients'].apply(remove_punctuation).apply(remove_stopwords)
df['TranslatedInstructions'] = df['TranslatedInstructions'].apply(remove_punctuation).apply(remove_stopwords)


In [42]:
df['TranslatedInstructions']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                         begin making Masala Karela Recipe deseed karela slice remove skin skin nutrients Add karela pressure cooker 3 tablespoon water salt turmeric powder pressure cook three whistles Release pressure immediately open lids Keep asideHeat oil heavy bottomed pan kadhai Add cumin seeds let sizzleOnce cumin seeds sizzled add onions saute till turns golden brown colorAdd karela red chilli powder amchur powder coriander powder besan Stir combine masalas karelaDrizzle little extra oil top mix Cover pan simmer Masala Karela stirring occasionally every

#### Named Entity Recognition

In [43]:
from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package brown to /home/amanv/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/amanv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/amanv/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/amanv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
import numpy as np
def get_word_postag(word):
    if pos_tag([word])[0][1].startswith('J'):
        return wordnet.ADJ
    if pos_tag([word])[0][1].startswith('V'):
        return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN
    
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    postag = get_word_postag(word)
    word = lemmatizer.lemmatize(word,postag)
    return word

#Extract noun
cols = ['TranslatedIngredients','TranslatedInstructions']

for col in cols:
    df['Noun_keywords_' + col] = np.nan
    for i in df.index:
      try:
        blob = TextBlob(df[col][i])  
        potential_searches = []
        if (df[col][i] != ""):
          for nouns in blob.noun_phrases:
              potential_searches.append(normalise(nouns))

          df['Noun_keywords_' + col][i] = potential_searches
        else:
          # potential_searches = []
          df['Noun_keywords_' + col][i] = potential_searches
      except:
        continue

/tmp/ipykernel_28025/709383043.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Noun_keywords_' + col][i] = potential_searches
/tmp/ipykernel_28025/709383043.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Noun_keywords_' + col][i] = potential_searches


#### To extract list of ingridients for fine tuning rasa

In [45]:

list_of_list = list(df['Noun_keywords_TranslatedIngredients'])


In [46]:
strings = []

for sublist in list_of_list:
    # Iterate through each string in the sublist
    for string in sublist:
        
        list_of_str=string.split(" ")
        # Split the string on the comma and add the resulting strings to the list
        strings.extend(list_of_str)


# Print the final list of strings
print(len(strings))




226001


#### Stored in a file named ingredients.txt

In [47]:
with open('ingredients.txt', 'w') as file:
    # Iterate through each string in the list and write it to a new line in the file
    for string in strings:
        file.write("        - "+string + '\n')